In [2]:
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer

def select_random_item_number():
    # Load the data from the CSV file
    data = pd.read_csv('Daily Snapshot.csv')

    # Check if the 'Item Number – 8 digit' column exists
    if 'Item Number – 8 digit' in data.columns:
        # Select a random 'Item Number – 8 digit'
        random_item_number = random.choice(data['Item Number – 8 digit'].tolist())
        return random_item_number

def ExactDrugAlgoFunction(drug_code):
    # Item number of the drug to run the similarity test on
    reference_item_number = drug_code

    # Let's open and read the content of the uploaded file to understand its structure and data.
    file_path = 'Daily Snapshot.csv'

    # Reading the file

    # Load the data from the CSV file
    data = pd.read_csv(file_path)

    # Removing the specified columns and reordering the 'Item Number – 8 digit' column
    columns_to_remove = ['Item Number – 6 digit', 'UPC Number', 'Constant', 
                        'Customer-Specific Item Number', 'Pack Size Divisor', 
                        'RX/OTC Indicator']

    # Removing the columns
    data_cleaned = data.drop(columns=columns_to_remove)

    # Reordering 'Item Number – 8 digit' to the left
    column_to_move = data_cleaned.pop('Item Number – 8 digit')
    data_cleaned.insert(0, 'Item Number – 8 digit', column_to_move)

    # Moving all price columns and the contract flag to the right
    columns_to_move = ['AWP Price', 'Acquisition Price', 'Retail Price', 'WAC Price', 'Contract Flag']
    for col in columns_to_move:
        data_cleaned[col] = data_cleaned.pop(col)

    import re

    # Function to split the generic description into generic name and form
    def split_description(desc):
        match = re.search(r'[A-Z]', desc)
        if match:
            index = match.start()
            return desc[:index].strip(), desc[index:].strip()
        else:
            return desc, ''

    # Applying the function to split 'Generic Description'
    data_cleaned['Generic Name'], data_cleaned['Form'] = zip(*data_cleaned['Generic Description'].apply(split_description))
    data_cleaned.drop(columns=['Generic Description'], inplace=True)

    # Removing rows where 'Generic Name' is empty or whitespace
    data_cleaned = data_cleaned[data_cleaned['Generic Name'].str.strip() != '']

    # Function to split the description into name and size
    def split_description_on_number(desc):
        match = re.search(r'\d', desc)
        if match:
            index = match.start()
            return desc[:index].strip(), desc[index:].strip()
        else:
            return desc, ''

    # Applying the function to split 'Description'
    data_cleaned['Name'], data_cleaned['Size'] = zip(*data_cleaned['Description'].apply(split_description_on_number))
    data_cleaned.drop(columns=['Description'], inplace=True)

    # Find and print the row for the given reference_item_number
    matching_row = data_cleaned[data_cleaned['Item Number – 8 digit'] == reference_item_number]
    

    # Find the Generic Name for the given reference_item_number
    reference_generic_name = data_cleaned.loc[data_cleaned['Item Number – 8 digit'] == reference_item_number, 'Generic Name'].iloc[0]


    # Create a copy of the dataframe filtered by Generic Name
    data_lidocaine = data_cleaned[data_cleaned['Generic Name'] == reference_generic_name].copy()

    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    # Selecting the reference item
    reference_item = data_lidocaine[data_lidocaine['Item Number – 8 digit'] == reference_item_number]
    if reference_item.empty:
        return "Reference item not found in the dataset."

    # Extracting the form of the reference item
    reference_form = reference_item.iloc[0]['Form']
    forms = data_lidocaine['Form'].tolist()
    forms.insert(0, reference_form)

    # Vectorizing the forms using TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(forms)

    # Calculating cosine similarity
    cosine_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:]).flatten()
    data_lidocaine['Similarity'] = cosine_similarities

    # Filtering the dataset to show only items with a similarity score above 0.9
    similarity_items = data_lidocaine[data_lidocaine['Similarity'] > 0].copy()

    # Extracting the size of the reference item
    reference_size = reference_item.iloc[0]['Size']
    sizes = similarity_items['Size'].tolist()
    sizes.insert(0, reference_size)

    # Vectorizing the sizes using TF-IDF
    tfidf_matrix_sizes = vectorizer.fit_transform(sizes)

    # Calculating cosine similarity for sizes
    cosine_similarities_sizes = cosine_similarity(tfidf_matrix_sizes[0:1], tfidf_matrix_sizes[1:]).flatten()
    similarity_items['Size Similarity'] = cosine_similarities_sizes

    # Filtering by size similarity
    high_similarity_items_filtered_by_size = similarity_items[similarity_items['Size Similarity'] > 0]

    # Remove the first row
    high_similarity_items_filtered_by_size = high_similarity_items_filtered_by_size.iloc[1:]
    
    high_similarity_items_filtered_by_size = high_similarity_items_filtered_by_size.sort_values(by=['Similarity'], ascending=False)
    high_similarity_items_filtered_by_size_2 = high_similarity_items_filtered_by_size.sort_values(by=['Size Similarity'], ascending=False)
    return high_similarity_items_filtered_by_size, matching_row

# Example usage
# results = ExactDrugAlgoFunction(your_drug_code)
# print(results.head())


In [8]:
#random_item_number = select_random_item_number()
data, input = ExactDrugAlgoFunction(10055750)

In [9]:
w1 = 2
w2 = 3
data['True Similarity'] = (w1 * data['Similarity'] + w2 * data['Size Similarity']) / (w1 + w2)

data = data.sort_values(by=['True Similarity'], ascending=False)
data = data.drop(columns=['Similarity', 'Size Similarity'])
data = data[data['Item Number – 8 digit'] != input['Item Number – 8 digit'].iloc[0]]

filtered = data[data['True Similarity'] > 0.5]

filtered.head(10)


,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form,Name,Size,True Similarity
37121,10056785,00904567761,100,1,29366,2309,0,2500,N,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 100 UD,0.894149
15933,10040999,68382009801,100,1,27330,482,0,1378,C,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 100,0.781140
11728,10031159,60505008302,100,1,27330,588,0,1387,C,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 100,0.781140
36358,10055752,68084004601,100,1,27824,4281,0,9482,C,paroxetine,HCl ORAL TABLET 30,PAROXETINE,30 MG TAB 100 UD AH,0.674763
36355,10055742,68084004401,100,1,26721,3708,0,8500,C,paroxetine,HCl ORAL TABLET 10,PAROXETINE,10 MG TAB 100 UD AH,0.650237
36360,10055753,68084004701,100,1,30344,4710,0,7800,C,paroxetine,HCl ORAL TABLET 40,PAROXETINE,40 MG TAB 100 UD AH,0.642926
35781,10055128,60505008301,30,1,8199,295,0,416,C,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 30,0.635821
33519,10190958,60505451803,30,1,33552,25815,0,27959,N,paroxetine,HCl ORAL TABLET 20,PAXIL,20 MG TAB 30,0.635821
19179,10180545,43547034803,30,1,8000,647,0,700,N,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 30,0.635821
7494,10021339,00378700293,30,1,8199,385,0,416,N,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 30,0.635821


In [102]:
input

,Item Number – 8 digit,NDC Number,Size Qty,Retail Pack Quantity,AWP Price,Acquisition Price,Retail Price,WAC Price,Contract Flag,Generic Name,Form,Name,Size
36356,10055750,68084004501,100,1,27888,1371,0,3231,C,paroxetine,HCl ORAL TABLET 20,PAROXETINE,20 MG TAB 100 UD AH
